In [ ]:
# Load environment variables and initialize Anthropic client for web search integration
# This cell sets up the SDK and loads API credentials from .env file

from dotenv import load_dotenv
from anthropic import Anthropic

# Load environment variables (including ANTHROPIC_API_KEY) from .env file
load_dotenv()

# Initialize the Anthropic client with credentials from environment
client = Anthropic()

# Specify the model to use for API requests - Sonnet 4.5 supports web search tool
model = "claude-sonnet-4-5"


In [ ]:
# Helper functions for managing conversations and API interactions
# These utilities simplify message handling and API calls with web search support

from anthropic.types import Message


def add_user_message(messages, message):
    """
    Add a user message to the conversation history.
    
    Parameters:
    - messages: List of message dictionaries representing conversation history
    - message: Either a string or Anthropic Message object containing user input
    
    Returns: None (modifies messages list in place)
    
    Note: Handles both string and Message object inputs for flexibility
    """
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    """
    Add an assistant message to the conversation history.
    
    Parameters:
    - messages: List of message dictionaries representing conversation history
    - message: Either a string or Anthropic Message object containing assistant response
    
    Returns: None (modifies messages list in place)
    
    Note: Converts Anthropic Message objects to content blocks for consistency
    """
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
    """
    Send a message to Claude and receive a response, with optional web search support.
    
    Parameters:
    - messages: List of message dictionaries representing conversation history
    - system: Optional system prompt to guide model behavior
    - temperature: Controls randomness of response (1.0 = default, lower = more deterministic)
    - stop_sequences: Optional list of strings to stop generation at
    - tools: Optional list of tool definitions (e.g., web_search_schema) to enable tool use
    
    Returns: Anthropic Message object containing the model's response
    
    The function builds parameters dynamically, only including tools and system 
    prompt if provided. Web search can be enabled by passing web_search_schema in tools.
    """
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    # Add tools parameter if tools are provided (enables web search or other tools)
    if tools:
        params["tools"] = tools

    # Add system prompt if provided
    if system:
        params["system"] = system

    # Call the Anthropic API with the constructed parameters
    message = client.messages.create(**params)
    return message


def text_from_message(message):
    """
    Extract plain text content from an Anthropic Message object.
    
    Parameters:
    - message: Anthropic Message object with content blocks
    
    Returns: String containing all text blocks joined by newlines
    
    Note: Filters out non-text blocks (e.g., tool_use blocks) and returns only text content
    """
    return "\n".join([block.text for block in message.content if block.type == "text"])


In [ ]:
# Define web search tool schema for Claude to use
# This enables Claude to perform searches on restricted domains (in this case, nih.gov)

web_search_schema = {
    "type": "web_search_20250305",  # Tool type: web search capability available in newer Claude models
    "name": "web_search",            # Tool name for model to reference in tool_use blocks
    "max_uses": 5,                   # Limit searches to 5 per request to control costs
    "allowed_domains": ["nih.gov"],  # Restrict searches to nih.gov domain only for health information
}


In [ ]:
# Test web search tool with a question about exercise and leg muscle building
# Demonstrates how Claude can perform web searches to find factual, current information

# Initialize conversation message history
messages = []

# Add user query about exercise recommendations
# Claude will use the web_search tool to find current information from nih.gov
add_user_message(
    messages,
    """
    What's the best exercise for gaining leg muscle?
    """,
)

# Send message to Claude with web_search tool enabled
# Claude can now perform up to 5 searches on nih.gov to answer the question
response = chat(messages, tools=[web_search_schema])

# Display the response (includes search results and Claude's analysis)
response
